# 모듈 6: 세션 영속성을 위한 데이터베이스 통합

이 모듈에서는 에이전트의 세션(Session)을 데이터베이스에 저장하여, 애플리케이션이 재시작되어도 대화 내용과 상태가 유지되도록 하는 방법을 배운다.

## 학습 목표

이 모듈을 완료하면 다음 내용을 이해할 수 있다.

- **DatabaseSessionService**: 데이터베이스를 사용하는 세션 서비스 설정 방법을 익힌다.
- **SQLite 연동**: 로컬 SQLite 데이터베이스에 대화 기록과 상태를 저장한다.
- **상태 접근 도구(Stateful Tools)**: `ToolContext`를 사용하여 세션 상태를 읽고 쓰는 도구를 만든다.
- **영속성 확인**: 애플리케이션을 재시작해도 에이전트가 이전 대화를 기억하는지 확인한다.

## 왜 데이터베이스 통합이 필요한가

이전 모듈에서 사용한 `InMemorySessionService`는 메모리(RAM)에만 데이터를 저장하기 때문에 프로그램이 종료되면 모든 기억이 사라진다. 실제 서비스에서는 다음과 같은 이유로 데이터베이스 통합이 필수적이다.

- **데이터 보존**: 서버 재시작이나 배포 후에도 사용자 경험을 유지해야 한다.
- **확장성**: 여러 서버 인스턴스가 동일한 세션 데이터를 공유해야 한다.
- **분석**: 대화 로그를 저장하여 추후 분석이나 개선에 활용할 수 있다.

In [22]:
# 필요한 패키지 설치
# sqlalchemy의 비동기 기능을 위해 greenlet과 aiosqlite가 필수적
%pip install -q google-adk python-dotenv sqlalchemy greenlet aiosqlite


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/miniconda3/envs/graph/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import uuid
from dotenv import load_dotenv

from google.adk import Agent
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from google.adk.tools import ToolContext

load_dotenv()
MODEL = "gemini-2.5-flash"

True

## 상태에 접근하는 도구 만들기 (ToolContext 활용)

도구(함수)가 현재 세션의 상태(State)를 읽거나 수정하려면 `tool_context: ToolContext` 파라미터를 인자로 받아야 한다. ADK는 이 타입 힌트를 인식하여 함수 호출 시 자동으로 컨텍스트 객체를 주입해준다.

**주의**: `tool_context` 파라미터는 에이전트(LLM)가 아닌 시스템이 사용하는 값이므로, Docstring(설명)에는 포함하지 않는 것이 좋다.

In [34]:
def add_reminder(reminder_text: str, tool_context: ToolContext) -> dict:
    """
    사용자의 리마인더 목록에 새 리마인더를 추가합니다.
    
    Args:
        reminder_text: 추가할 할 일 내용
    """
    # tool_context.state를 통해 세션 상태에 접근
    state = tool_context.state
    # 기존 리마인더 가져오기 (없으면 빈 리스트)
    reminders = state.get("reminders", [])
    
    reminders.append(reminder_text)
    
    # 상태 업데이트 (중요: 변경된 객체를 다시 할당해야 함)
    state["reminders"] = reminders
    
    return {"status": "success", "message": f"리마인더 추가됨: {reminder_text}"}

def view_reminders(tool_context: ToolContext) -> list:
    """
    현재 저장된 모든 리마인더 목록을 조회합니다.
    """
    state = tool_context.state
    return state.get("reminders", [])

def delete_reminder(index: int, tool_context: ToolContext) -> dict:
    """
    특정 인덱스의 리마인더를 삭제합니다.
    
    Args:
        index: 삭제할 항목의 번호 (0부터 시작)
    """
    state = tool_context.state
    reminders = state.get("reminders", [])
    
    if 0 <= index < len(reminders):
        removed = reminders.pop(index)
        state["reminders"] = reminders
        return {"status": "success", "message": f"삭제됨: {removed}"}
    else:
        return {"status": "error", "message": "잘못된 인덱스입니다."}

## 리마인더 에이전트 생성하기

위에서 정의한 도구들을 사용하는 에이전트를 정의한다. 에이전트 지침(Instruction)에 현재 상태를 보여주도록 템플릿(`{reminders}`)을 사용할 수 있다.

In [35]:
memory_agent = Agent(
    name="memory_agent",
    model=MODEL,
    description="사용자의 할 일을 기억하는 리마인더 에이전트",
    instruction="""
    당신은 꼼꼼한 리마인더 관리 비서입니다.
    
    사용자가 할 일을 추가하거나, 조회하거나, 삭제 요청을 하면 적절한 도구를 사용하세요.
    사용자가 현재 리마인더 목록을 물어보거나 확인이 필요할 때는 'view_reminders' 도구를 사용하여 정보를 확인하세요.
    
    항상 작업 결과를 확인한 후 사용자에게 친절하게 알려주세요.
    """,
    tools=[add_reminder, view_reminders, delete_reminder]
)

## 데이터베이스 세션 서비스 설정

`DatabaseSessionService`를 사용하여 SQLite 데이터베이스에 연결한다. **중요**: ADK는 비동기 프레임워크이므로 `sqlite+aiosqlite:///` 스키마를 사용해야 한다.

In [36]:
# SQLite 데이터베이스 세션 서비스 생성
# 'db_url' 파라미터에 비동기 드라이버(aiosqlite) 스키마를 사용
session_service = DatabaseSessionService(
    db_url="sqlite+aiosqlite:///agent_sessions.db"
)

print("데이터베이스 세션 서비스가 준비되었습니다.")

데이터베이스 세션 서비스가 준비되었습니다.


## 세션 관리 및 Runner 생성

이제 `Runner`에 데이터베이스 세션 서비스를 연결한다. **Runner 생성 시 `app_name`을 반드시 지정해야 한다.**

In [37]:
# Runner 생성 (DB 세션 서비스 주입)
runner = Runner(
    agent=memory_agent,
    app_name="ReminderApp", # 필수: 애플리케이션 이름
    session_service=session_service
)

# 테스트를 위해 고정된 세션 ID 사용 (영속성 확인용)
# 실제 앱에서는 사용자별로 고유한 ID를 생성해야 함
session_id = "persistent-session-v1"

# 세션이 없으면 새로 생성, 있으면 불러오기
session = await session_service.get_session(
    app_name="ReminderApp",
    user_id="user_1",
    session_id=session_id
)

if not session:
    print("새로운 세션을 생성합니다.")
    session = await session_service.create_session(
        app_name="ReminderApp",
        user_id="user_1",
        session_id=session_id,
        state={"reminders": []}  # 초기 상태
    )
else:
    print(f"기존 세션을 불러왔습니다. (ID: {session.id})")
    
    # 중요: 이전 실행 오류 등으로 인해 'reminders' 키가 없을 경우를 대비한 복구 로직
    if "reminders" not in session.state:
        print("경고: 세션 상태에 'reminders' 키가 누락되어 초기화합니다.")
        session.state["reminders"] = []
        await session_service.update_session(session)
    
    print(f"저장된 리마인더: {session.state.get('reminders')}")

기존 세션을 불러왔습니다. (ID: persistent-session-v1)
저장된 리마인더: []


## 에이전트 실행 테스트

`run_debug`를 사용하여 에이전트와 대화한다. 대화가 진행될수록 데이터베이스에 내용이 저장된다.

In [38]:
# 1. 리마인더 추가 요청
await runner.run_debug(
    "내일 오후 2시에 치과 예약이 있다고 기록해줘.", 
    session_id=session_id
)


 ### Continue session: persistent-session-v1

User > 내일 오후 2시에 치과 예약이 있다고 기록해줘.


memory_agent > 알겠습니다. 내일 오후 2시에 치과 예약, 마트에서 우유랑 계란 사기 이렇게 두 가지 리마인더를 추가했습니다. 현재 할 일 목록은 다음과 같습니다:

1. 내일 오후 2시에 치과 예약
2. 마트에서 우유랑 계란 사기



[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'reminder_text': '내일 오후 2시에 치과 예약'
         },
         id='adk-19931b4d-35a4-4081-99cc-aa0c88d83ac4',
         name='add_reminder'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=18,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=18
     ),
   ],
   prompt_token_count=529,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=529
     ),
   ],
   total_token_count=547
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, av

In [39]:
# 2. 또 다른 리마인더 추가
await runner.run_debug(
    "마트에서 우유랑 계란 사기도 추가해.", 
    session_id=session_id
)


 ### Continue session: persistent-session-v1

User > 마트에서 우유랑 계란 사기도 추가해.
memory_agent > 마트에서 우유랑 계란 사기를 리마인더 목록에 추가했습니다.



[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'reminder_text': '마트에서 우유랑 계란 사기'
         },
         id='adk-f58a332b-2524-4204-8854-b9e5f2ee4621',
         name='add_reminder'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=16,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=16
     ),
   ],
   prompt_token_count=734,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=734
     ),
   ],
   total_token_count=750
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg

### 영속성(Persistence) 확인

이제 커널을 재시작하거나 아래 코드를 다시 실행해도, `agent_sessions.db` 파일이 삭제되지 않는 한 에이전트는 위에서 추가한 리마인더를 기억하고 있어야 한다.

In [40]:
# 3. 리마인더 조회 요청
await runner.run_debug(
    "지금 내 할 일 목록이 뭐지?", 
    session_id=session_id
)


 ### Continue session: persistent-session-v1

User > 지금 내 할 일 목록이 뭐지?
memory_agent > 현재 리마인더 목록은 다음과 같습니다:
1. 내일 오후 2시에 치과 예약
2. 마트에서 우유랑 계란 사기
3. 마트에서 우유랑 계란 사기


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={},
         id='adk-d455b5e9-2bc8-48bd-96d7-26d032b01a23',
         name='view_reminders'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=4,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=4
     ),
   ],
   prompt_token_count=804,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=804
     ),
   ],
   total_token_count=808
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=1.1615920811891556e-06, logprobs_result=None,

## 정리

이 모듈에서는 **DatabaseSessionService**를 사용하여 대화 내용과 에이전트의 상태를 영구적으로 저장하는 방법을 배웠다.

1. **ToolContext**: 도구 함수에서 세션 상태(`state`)를 읽고 쓰는 방법을 익혔다.
2. **데이터베이스 연동**: `sqlite+aiosqlite:///` URI를 사용하여 비동기 DB 연결을 구현했다.
3. **상태 복구**: 세션 상태가 손상되었을 때(키 누락 등) 이를 감지하고 복구하는 로직을 추가했다.

이제 에이전트는 '기억'을 가지게 되었다. 다음 모듈에서는 이 기억을 바탕으로 더 복잡한 협업을 수행하는 멀티 에이전트 시스템을 다룰 것이다.